<a href="https://colab.research.google.com/github/GalvanLuis/NeuralNetClass/blob/main/AnimalClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai --upgrade

#Clasificador de animales

In [ ]:
import fastai.vision.all as fv
import torch 
from pathlib import Path
import pandas as pd 
from sklearn.model_selection import train_test_split
import shutil
import os

###Manejo de archivos
>Para comenzar con el entrenamiento de la red primero he de cargar los datos (imagenes y csv), el archivo csv contiene las etiquetas a las que pertenece cada imagen.

In [ ]:
labels = pd.DataFrame(pd.read_csv('/content/train.csv'))
labels

###Separacion de datos
>Ahora voy a asignar a los datos aleatoriamente "train" o "valid" para despues pasarlos a sus respectivas carpetas

In [ ]:
train, valid = train_test_split(labels, test_size = 0.33, random_state = 42) 
train.shape, valid.shape

In [ ]:
pics = Path("/content/drive/MyDrive/AnimalsSmall/AnimalsSmall/images")
(pics).ls()

>Ahora que ya estan asignadas las imagenes las mandare a sus respectivas carpetas, para esto, dentro de "train" y "valid" creare nuevas subcarpetas con el nombre de la etiqueta que tienen. Es decir, si no existe una carpeta con la etiqueta de la imagen que lee, supongamos "wolf", la creara y de alli en adelante metera todas las imagenes de "wolf"

###-Carpeta "train"

In [ ]:
tmp_train = [] 
for index, element in train.iterrows():
        tmp_train.append((f"/content/drive/MyDrive/AnimalsSmall/AnimalsSmall/images/{element[0]}", element[1]))

print(tmp_train)

In [ ]:
for path in range(len(img)):
    tmp_path = str(img[path])
    for element in tmp_train: 
        if tmp_path == element[0]:
            try:
                os.mkdir(f"/content/drive/MyDrive/AnimalsSmall/AnimalsSmall/train/{element[1]}") # Creamos la subcarpeta
                shutil.copy(img[path], f"/content/drive/MyDrive/AnimalsSmall/AnimalsSmall/train/{element[1]}/") #movemos 
            except:
                shutil.copy(img[path], f"/content/drive/MyDrive/AnimalsSmall/AnimalsSmall/train/{element[1]}/") #movemos
                



###-Carpeta "valid"

In [ ]:
tmp_valid = [] 
for index, element in valid.iterrows():
        tmp_valid.append((f"/content/images/{element[0]}", element[1]))
print(tmp_valid)

In [ ]:
for path in range(len(img)):
    tmp_path = str(img[path])
    for element in tmp_valid: 
        if tmp_path == element[0]:
            try:
                os.mkdir(f"/content/valid/{element[1]}") # Creamos la subcarpeta
                shutil.copy(img[path], f"/content/valid/{element[1]}/") #movemos 
            except:
                shutil.copy(img[path], f"/content/valid/{element[1]}/") #movemos


##**Entrenamiento**

> * Ahora realizare los procedimientos como los que se muestran en el video

>* Aplicare algunas transformaciones a las imagenes y algunos otros paramentros para comparar entre modelos

In [ ]:
animalitos = Path("/content/animalitos");!ls "/content/animalitos"

In [ ]:
files = fv.get_image_files(animalitos);files #Obtencion de archivos dentro de la carpeta y subcarpetas de "animalitos"


In [ ]:
def load(folder, size, batch_size):
  tfms = fv.aug_transforms(max_rotate= 180, 
                           max_lighting=0.80, 
                           max_zoom=1.2, 
                           max_warp=0.2)
  
  data = fv.DataBlock(blocks = (fv.ImageBlock, fv.CategoryBlock), 
                      get_items=fv.get_image_files, 
                      get_y=fv.parent_label, 
                      splitter=fv.GrandparentSplitter(),
                      item_tfms=fv.Resize(size), 
                      batch_tfms=tfms,)
  return data.dataloaders(folder, bs=batch_size)

In [ ]:
data = load(animalitos, size=128, batch_size=28) #Cargamos los archivos todos de tamaño 128x128 y el tamaño de nuestro batch es de 64

In [ ]:
data.show_batch() #Listo, ahora si se muestran jaja

In [ ]:
x,y = data.one_batch() #x tiene al tamaño de nuestro batch, los canales de color y las dimensiones de nuestras imagenes, y tiene el tamaño de nuestro batch
x.shape, y.shape

###Modelo1

####Ahora utilizare un modelo preentrenado para el entrenamiento de las ultimas capas

>*Nuestro modelo sera el "resnet18".
Learner sera nuestro primer intento el cual toma los datos, el modelo preentrenado y nuestra metrica sera "accuracy"

In [ ]:
learner = fv.cnn_learner(data, fv.resnet18, metrics=fv.accuracy) #Descargara los archivos del modelo preentrenado

In [ ]:
learner.lr_find() #Buscamos el learning rate

In [ ]:
learner.fit_one_cycle(2, lr_max=1e-2) #Lo manejare a 5 epocas

In [ ]:
learner.recorder.plot_sched()

In [ ]:
learner.save('stage1') #Guardare el modelo

In [ ]:
#learner.load('stage1') Lo dejo aqui por si acaso

####Ahora hay que entrenar todas las capas

In [ ]:
learner.unfreeze() #vamos a descongelar toda la red 

In [ ]:
learner.lr_find()

In [ ]:
learner.fit_one_cycle(4, lr_max=slice(1e-6, 2e-4)) #Ahora le learnig rate variara entre el rango que estoy poniendo

In [ ]:
learner.save('stage2')

In [ ]:
learner.show_results(ds_idx=3, dl=data.train, shuffle=True)

####Resultados del modelo 1

In [ ]:
interp = fv.ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.plot_top_losses(9)

###Modelo2

In [ ]:
def load2(folder, size, batch_size):
  tfms = fv.aug_transforms(max_rotate= 360,
                           flip_vert=True, 
                           max_lighting=0.25, 
                           max_zoom=2.2, 
                           min_zoom=0.6,
                           max_warp=0.7,
                           do_flip=False
                           )
  
  data = fv.DataBlock(blocks = (fv.ImageBlock, fv.CategoryBlock), 
                      get_items=fv.get_image_files, 
                      get_y=fv.parent_label, 
                      splitter=fv.GrandparentSplitter(),
                      item_tfms=fv.Resize(size), 
                      batch_tfms=tfms,)
  return data.dataloaders(folder, bs=batch_size)

In [ ]:
data2 = load2(animalitos, size=128, batch_size=100)

In [ ]:
x2,y2 = data2.one_batch()

####Entrenamiento de las ultimas capas del modelo 2

In [ ]:
learner2 = fv.cnn_learner(data2, fv.resnet18, metrics=fv.accuracy)

In [ ]:
learner2.lr_find()

In [ ]:
learner2.fit_one_cycle(20, lr_max=0.0012022644514217973)

In [ ]:
learner2.recorder.plot_sched()

In [ ]:
learner2.save('stage1_model2')

####Entrenamiento de todas las capas del modelo 2

In [ ]:
learner2.unfreeze() # Ahora debdemos de tener cuidado 

In [ ]:
learner2.lr_find()

In [ ]:
learner2.fit_one_cycle(4, lr_max=slice(1e-06, 2e-4)) #Entrenamos con varias learning rates

In [ ]:
learner2.save('stage2_model2')

In [ ]:
learner2.show_results(ds_idx=3, dl=data2.train, shuffle=True)

###Modelo 3

In [ ]:
def load3(folder, size, batch_size):
  tfms = fv.aug_transforms(max_rotate= 45,
                           flip_vert=True, 
                           max_lighting=0.40, 
                           max_zoom=1.2, 
                           min_zoom=0.2,
                           max_warp=0.2,
                           do_flip=True
                           )
  
  data = fv.DataBlock(blocks = (fv.ImageBlock, fv.CategoryBlock), 
                      get_items=fv.get_image_files, 
                      get_y=fv.parent_label, 
                      splitter=fv.GrandparentSplitter(),
                      item_tfms=fv.Resize(size), 
                      batch_tfms=tfms,)
  return data.dataloaders(folder, bs=batch_size)

In [ ]:
data3 = load3(animalitos, size=128, batch_size=128)

In [ ]:
x3,y3 = data3.one_batch()

Entrenamiento de las ultimas capas del modelo 3

In [ ]:
learner3 = fv.cnn_learner(data3, fv.resnet101, metrics=fv.accuracy)

In [ ]:
learner3.lr_find()

In [ ]:
learner3.fit_one_cycle(3, wd=0.1, div=1, div_final=100, pct_start=0.6, lr_max=slice(0.0005754399462603033, 1e-2)) 

In [ ]:
learner3.recorder.plot_sched()

In [ ]:
learner3.save('stage1_model3')

In [ ]:
learner3.show_results(ds_idx=3, dl=data3.train, shuffle=True)

####Entrenamiento de las ultimas capas de modelo 3

In [ ]:
learner3 = fv.cnn_learner(data3, fv.resnet18, metrics=fv.accuracy)

In [ ]:
learner3.lr_find()

In [ ]:
learner3.fit_one_cycle(3, lr_max=1e-3) 

In [ ]:
learner3.recorder.plot_sched()

In [ ]:
learner3.save('stage1_model3')

####Resultados del entrenamiento sin todas las capas

In [ ]:
learner3.show_results(ds_idx=3, dl=data3.train, shuffle=True)

In [ ]:
interp = fv.ClassificationInterpretation.from_learner(learner3)

In [ ]:
interp.plot_top_losses(9)

Notas


In [ ]:
interp.plot_confusion_matrix(dpi = 90,figsize = (6,6))

###Modelo 4
Con otro resnet

###Modelo 5
Muchas epocas 

###Modelo Final
High accuracy 20 epocas, resnet 1 y parametros del moche